In [98]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F

In [4]:
sc = SparkContext.getOrCreate()
print(sc)

In [ ]:
# join 
# ip <-> ar <-> mb
#           <-> dept

# ip <-> ar : "ip_id"
# ip <-> cc_cst : "x"
# cc_cst <-> cc_visa : "x"
# ar <-> mb : "src_ar_id"
# ar <-> dept : "ar_id"

# DerivedFeature
# [PerWeek & PerMonth]
# noTransferInUnique, noTransferOutUnique, noTransferIn, noTransferOut, ratioTransferIn, ratioTransferOut
# avgTransferInAmount, avgTransferOutAmount, avgTransferAmount
# noFeeAmountGroupBy(0,10,25,35)
# noTransferInPeriod(morning,afternoon,evening,night :: 6hr)
# noTransferOutPeriod(morning,afternoon,evening,night :: 6hr)
# noTransferInPerDay(Mon-Thu, Fri, Sat-Sun [avg]) <-- PerMonth
# noTransferOutPerDay(Mon-Thu, Fri, Sat-Sun [avg]) <-- PerMonth
# avgTransferInHoliday, avgTransferOutHoliday


# Deposit
# [PerWeek & PerMonth]
# noBranchVisit,noBranchVisitUnique,ratioBranchVisit
# avgTransferInAmount, avgTransferOutAmount, avgTransferAmount
# noDepositAmount, noWithdrawAmount
# avgDepositAmount, avgWithdrawAmount
# noWithdrawPeriod(morning,afternoon,evening,night :: 6hr)
# noDepositPeriod(morning,afternoon,evening,night :: 6hr)
# noDepositPerDay(Mon-Thu, Fri, Sat-Sun [avg]) <-- PerMonth
# noWithdrawPerDay(Mon-Thu, Fri, Sat-Sun [avg]) <-- PerMonth
# noTransferInUnique, noTransferOutUnique, noTransferIn, noTransferOut, ratioTransferIn, ratioTransferOut
# noTransferInPerDay(Mon-Thu, Fri, Sat-Sun [avg]) <-- PerMonth
# noTransferOutPerDay(Mon-Thu, Fri, Sat-Sun [avg]) <-- PerMonth
# ??Balance??
# ??CBS Sub Operation Code??
# ahrzd_usr_id <-- [KMP, EDC, ATM]

# Credit


# Approach
# 1 Detech ลูกน้องโดยดูจำนวนเงินที่รับมากที่สุดแล้วเช็ค

# Visualize
# Location

# feature = ["vc_ip.ip_id"]
# category = ["vc_ip.ip_tp_cd","vc_ip.mar_st_cd","vc_ip.ctf_tp_cd","vc_ip.ocp_cd","vc_ip.idv_incm_seg_cd"]
# filteredFeature = ["vc_ip.prvn_f","vc_ip.ip_st_cd","vc_ip.death_f"]

In [6]:
spark = (SparkSession
         .builder
         .appName("MerchantInsight")
         .config("spark.sql.warehouse.dir", "/opt/jupyter_workspace/spark-warehouse")
         .getOrCreate())
print (spark)

In [30]:
mb_trans_data =(spark
      .read
      .option("header", "true")
      .option("inferSchema","true")
      .csv('dataset/mock-transaction.csv'))

In [31]:
mb_trans_data.show()

+--------+--------+--------+------+------+------+-------+
|fm_ar_id|to_ar_id|ptn_yyyy|ptn_mm|ptn_dd|ptn_hr|ptn_min|
+--------+--------+--------+------+------+------+-------+
|     128|     140|    2016|     7|     6|    17|      1|
|     114|     150|    2016|     7|     6|    17|      1|
|     131|     120|    2016|     7|     6|    17|      1|
|     144|     104|    2016|     7|     6|    17|      1|
|     133|     130|    2016|     7|     6|    17|      1|
|     109|     140|    2016|     7|     6|    17|      1|
|     116|     146|    2016|     7|     6|    17|      1|
|     110|     128|    2016|     7|     6|    17|      1|
|     101|     123|    2016|     7|     6|    17|      1|
|     128|     137|    2016|     7|     6|    17|      1|
|     108|     114|    2016|     7|     6|    17|      1|
|     103|     142|    2016|     7|     6|    17|      1|
|     135|     123|    2016|     7|     6|    17|      1|
|     118|     141|    2016|     7|     6|    17|      1|
|     138|    

In [81]:
#Pre-processing data
def getQuarter(dd,mm):
    m31 = [1,3,5,7,8,10,12]
    m30 = [4,6,9,11]
    if(mm in m31):
        if(dd in range(1,9)):
            return 1
        elif(dd in range(9,16)):
            return 2
        elif(dd in range(16,24)):
            return 3
        else:
            return 4
    elif(mm in m30):
        if(dd in range(1,9)):
            return 1
        elif(dd in range(9,16)):
            return 2
        elif(dd in range(16,23)):
            return 3
        else:
            return 4
    else:
        return dd/4

getQuarterUdf = udf(getQuarter,IntegerType()) 

In [110]:
mb_trans_data = mb_trans_data.withColumn("quarter",getQuarterUdf(mb_trans_data['ptn_dd'],mb_trans_data['ptn_mm']))
mb_trans_data.show()

+--------+--------+--------+------+------+------+-------+-------+
|fm_ar_id|to_ar_id|ptn_yyyy|ptn_mm|ptn_dd|ptn_hr|ptn_min|quarter|
+--------+--------+--------+------+------+------+-------+-------+
|     128|     140|    2016|     7|     6|    17|      1|      1|
|     114|     150|    2016|     7|     6|    17|      1|      1|
|     131|     120|    2016|     7|     6|    17|      1|      1|
|     144|     104|    2016|     7|     6|    17|      1|      1|
|     133|     130|    2016|     7|     6|    17|      1|      1|
|     109|     140|    2016|     7|     6|    17|      1|      1|
|     116|     146|    2016|     7|     6|    17|      1|      1|
|     110|     128|    2016|     7|     6|    17|      1|      1|
|     101|     123|    2016|     7|     6|    17|      1|      1|
|     128|     137|    2016|     7|     6|    17|      1|      1|
|     108|     114|    2016|     7|     6|    17|      1|      1|
|     103|     142|    2016|     7|     6|    17|      1|      1|
|     135|

In [91]:
getQuarter(16,5)

3

In [114]:
# mb_trans_data.select('fm_ar_id').groupby('fm_ar_id').count().show()
mb_trans_data.select('fm_ar_id',mb_trans_data['quarter']).rollup('fm_ar_id',mb_trans_data['quarter']).count().show()
# F.count(mb_trans_data['fm_ar_id']==148).alias("count148")

+--------+-------+-----+
|fm_ar_id|quarter|count|
+--------+-------+-----+
|     108|   null|   10|
|     118|   null|    7|
|     105|   null|    4|
|     144|      1|    9|
|     139|   null|    7|
|     109|      1|    5|
|     117|   null|    6|
|     125|      1|    3|
|     142|   null|    7|
|     120|   null|    6|
|     121|   null|    5|
|     112|   null|    6|
|     109|   null|    5|
|     129|   null|   11|
|     126|      1|    5|
|     137|      1|    4|
|     127|      1|    4|
|     134|   null|   12|
|    null|   null|  299|
|     136|   null|    6|
+--------+-------+-----+
only showing top 20 rows



## Features Extraction

In [47]:
#Feature Extraction
def getMbFrequency(columnName,newColumnName):
    freqIn = mb_trans_data.select(columnName).groupby(columnName).count()
    if((columnName == "fm_ar_id") | (columnName == "to_ar_id")):
        freqIn = freqIn.withColumnRenamed(columnName,"ar_id")
    freqIn = freqIn.withColumnRenamed("count",newColumnName)
    return freqIn

In [92]:
df = mb_trans_data.select("fm_ar_id").distinct().withColumnRenamed("fm_ar_id","ar_id")
train_data = df.join(getMbFrequency("fm_ar_id","noMbTransferOut"),"ar_id","left_outer")
train_data = train_data.join(getMbFrequency("to_ar_id"," noMbTransferIn"),"ar_id","left_outer")
train_data.show()

+-----+---------------+---------------+
|ar_id|noMbTransferOut| noMbTransferIn|
+-----+---------------+---------------+
|  148|              7|              3|
|  137|              4|              7|
|  133|             10|              6|
|  108|             10|              6|
|  101|              6|              4|
|  126|              5|              7|
|  115|              3|              5|
|  103|              3|              6|
|  128|             11|              5|
|  122|              2|              9|
|  140|              3|              6|
|  132|              7|              5|
|  146|              5|              6|
|  142|              7|              4|
|  139|              7|              4|
|  120|              6|              5|
|  117|              6|              6|
|  112|              6|              8|
|  127|              4|              5|
|  107|              6|             11|
+-----+---------------+---------------+
only showing top 20 rows

